# Imports and constants

In [ ]:
!pip install -U sentence-transformers
!pip install bert-extractive-summarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# For extractive summarization and sentence tokenization
import nltk
nltk.download('punkt')
from sentence_transformers import SentenceTransformer, util

# For compute the degree_centrality_scores
from scipy.sparse.csgraph import connected_components
from scipy.special import softmax
import logging

# For compute an abstractive summarize
from summarizer import Summarizer
from summarizer.sbert import SBertSummarizer

# Utils
import numpy as np
from collections import defaultdict
import copy
import string
from tqdm import tqdm
import os
import json
import csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Unzip the FairySum.zipper file and set the path of fairy tales and stories as constant.

In [ ]:
!unzip FairySum.zip
FAIRY_TALE = "./FAIRY_TALE/texts/"
SHORT_STORY = "./SHORT_STORY/texts/"

Archive:  FairySum.zip
  inflating: SHORT_STORY/texts/bn_02464343n_From Beyond.txt  
  inflating: SHORT_STORY/texts/bn_02975525n_Rothschild_s Violin.txt  
  inflating: SHORT_STORY/texts/bn_02413967n_The Black Cat.txt  
  inflating: SHORT_STORY/texts/bn_01745953n_An Occurrence at Owl Creek Bridge.txt  
  inflating: SHORT_STORY/texts/bn_03520917n_The Transition of Juan Romero.txt  
  inflating: SHORT_STORY/texts/bn_00210737n_Alyosha the Pot.txt  
  inflating: SHORT_STORY/texts/bn_03149445n_The Tree.txt  
  inflating: SHORT_STORY/texts/bn_01323408n_The Damned Thing.txt  
  inflating: SHORT_STORY/texts/bn_01789382n_Rip Van Winkle.txt  
  inflating: SHORT_STORY/texts/bn_03520887n_The Evil Clergyman.txt  
  inflating: SHORT_STORY/texts/bn_03297228n_.007.txt  
  inflating: SHORT_STORY/texts/bn_02485046n_Brooksmith.txt  
  inflating: SHORT_STORY/texts/bn_01304399n_The Music of Erich Zann.txt  
  inflating: SHORT_STORY/texts/bn_03220342n_The Rats in the Walls.txt  
  inflating: SHORT_STORY/text

# Functions

###My functions

In [ ]:
def pprint(obj):
    '''
        Indented print of an object
    '''
    print(json.dumps(obj, indent=3))

def read_story(story_path):
    '''
        Read a story.
        Input:
            story_path:
                path of the file.
        output:
            id2sentence:
                dictionary number_of_the_row -> sentence.
            sentence2id:
                dictionary sentence -> number_of_the_row.
            whole_story:
                the whole story in a single row string.
    '''
    with open(story_path, "r") as story:
        id2sentence = {}
        sentence2id = {}
        whole_story = ""
        for sentence in story:
            row_number = sentence.split(":")[0]
            row_text = ":".join(sentence.split(":")[1:]).strip()
            whole_story += row_text + " "
            id2sentence[row_number] = row_text
            sentence2id[row_text] = row_number
        whole_story = whole_story[:-1]
    return id2sentence, sentence2id, whole_story

def read_stories():
    '''
        Read the stories in the datasets.
        output:
            stories:
                dictionary that links for each story_name to another dictionary, 
                which has as keys: id2sentence, sentence2id and story 
                that are obtained from the read_story function.
    '''
    stories = defaultdict(lambda : {"id2sentence": "",
                                    "sentence2id": "",
                                    "story": ""})
    batch_pbar = tqdm(os.listdir(FAIRY_TALE), total=len(os.listdir(FAIRY_TALE)))
    for story_name in batch_pbar:
        batch_pbar.set_postfix({'Read Fairy Tale': story_name})
        story_id2sentence, story_sentence2id, story = read_story(FAIRY_TALE + story_name)
        stories[story_name]["id2sentence"] = story_id2sentence
        stories[story_name]["sentence2id"] = story_sentence2id
        stories[story_name]["story"] = story
    batch_pbar = tqdm(os.listdir(SHORT_STORY), total=len(os.listdir(SHORT_STORY)))
    for story_name in batch_pbar:
        batch_pbar.set_postfix({'Read Short Story': story_name})
        story_id2sentence, story_sentence2id, story = read_story(SHORT_STORY + story_name)
        stories[story_name]["id2sentence"] = story_id2sentence
        stories[story_name]["sentence2id"] = story_sentence2id
        stories[story_name]["story"] = story
    return stories

def get_original_sentence(original_sentences, left_sentences, generated_sentence):
    '''
        Given a generated sentence, get the one present in the original story 
        without the changes caused by the abstractive summarizer tokenized with a sentence tokenizer.
        To do this, I did a word-by-word check of the generated sentence. 
        If the word is present in one of the "left_sentences" sentences 
        then it is kept and the check is repeated with the next word. 
        This is repeated until the sentence ends or only one sentence, 
        the correct one, is left in the set of "left_sentences".
        input:
            original_sentences:
                A list of sentences found in the original story that were not summarized.
            left_sentences:
                Set of sentences that will be skimmed slowly. Initially it is the same as original_sentences.
            generated_sentence:
                A sentence generated from the abstractive summarizer tokenized with a sentence tokenizer.
        output:
            None, if the original sentence cannot be recognized by the generated one.
            left_sentences.pop(), otherwise:
                this is the only sentence kept after the process that is the original one to be selected.
    '''
    # This is a special case where the generated sentence contains three original sentences.
    if generated_sentence == 'The cock answered, "On Mr. Korbes a call to pay, And that is where we go to-day!"':
        # Then return the ids of the original sentences.
        # Since this is a single case, I decided to handle it in a non-algorithmic way.
        return ["4", "5", "6"]
    word_i = 0
    words = generated_sentence.split(" ")
    # Removes all words that are not alphanumeric, but words with some non-alphanumeric characters are retained.
    words = [word if any(j.isalnum() for j in word) else ''.join(e for e in word if e.isalnum()) for word in words]
    # After this check, some words may be removed and replaced with "". The words "" are not considered
    for i in range(words.count("")):
        words.remove("")
    # If all the words are "", then the sentence is something non-alphanumeric and is ignored.
    if words == []:
        return None
    # The skimming procedure
    while len(left_sentences) > 1 and word_i < len(words):
        word = words[word_i]
        sentence_delete = []
        # Check if the word is present in any original sentence and keeps it
        for original_sentence in original_sentences:
            if word not in original_sentence:
                sentence_delete.append(original_sentence)
        difference = left_sentences.difference(sentence_delete)
        # If the word is not present in all the original sentences, it is best to ignore it and move on to the next one.
        if difference != set():
            left_sentences = difference
        word_i += 1
    # Original sentence not found
    if len(left_sentences) == 0 or len(left_sentences) > 1:
        return None
    # Original sentence found
    else:
        return left_sentences.pop()

def restore_sentences(story, generated_sentences, story_name):
    '''
        Restore the sentences present in the original story given 
        the abstractive summarizer tokenized with a sentence tokenizer.
        input:
            story:
                Original story dictionary with id2sentence, sentence2id, story. Look read_stories() function.
            generated_sentences:
                Sentences generated by the abstractive summarizer tokenized with a sentence tokenizer.
            story_name:
                Name of the story .txt
        output:
            sentences_out:
                A list of original sentences restored, which is an extractive summarize.
    '''
    sentences_out = []
    original_sentences = list(story["id2sentence"].values())
    left_sentences = set(list(story["id2sentence"].values()))
    g = False
    for idx, generated_sentence in enumerate(generated_sentences):
        # If the generated sentence is present in the original sentence, just keep it.
        if story["sentence2id"].get(generated_sentence) is not None:
            sentences_out.append(generated_sentence)
        else:
            original_sentence = get_original_sentence(original_sentences, copy.deepcopy(left_sentences), generated_sentence)
            # Special case handled manually.
            if original_sentence == ["4", "5", "6"]:
                for sentence_id in original_sentence:
                    sentences_out.append(story["id2sentence"][sentence_id])
            # If the original sentence is not None, then we have found it.
            elif original_sentence is not None:
                sentences_out.append(original_sentence)
    return sentences_out

def generated_summary(stories):
    '''
        Generate the extractive summaries with nltk.sent_tokenize from the abstrct summaries.
        input:
            stories:
                All the original stories as dictionary with id2sentence, sentence2id, story. Look read_stories() function.
        output:
            nltk_summaries:
                List of sentences of the summaries.
    '''
    nltk_summaries = {}
    model = SBertSummarizer('paraphrase-MiniLM-L6-v2')
    batch_pbar = tqdm(stories.items(), total=len(list(stories.items())))
    for story_name, story_dict in batch_pbar:
        batch_pbar.set_postfix({'Generate summary of story': story_name})
        generated_sentences = nltk.sent_tokenize(model(stories[story_name]["story"]))
        nltk_summaries[story_name] = generated_sentences
    return nltk_summaries

### Functions from the paper
    title = "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks",
    author = "Reimers, Nils and Gurevych, Iryna",
    booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing",
    month = "11",
    year = "2019",
    publisher = "Association for Computational Linguistics",
    url = "https://arxiv.org/abs/1908.10084",

Compute the degree centrality scores.

This calculation allows for consideration of the n most important ("central") sentences in the text 

In [ ]:
"""
LexRank implementation
Source: https://github.com/crabcamp/lexrank/tree/dev
"""

logger = logging.getLogger(__name__)

def degree_centrality_scores(
    similarity_matrix,
    threshold=None,
    increase_power=True,
):
    if not (
        threshold is None
        or isinstance(threshold, float)
        and 0 <= threshold < 1
    ):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None',
        )

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(
            similarity_matrix,
            threshold,
        )

    scores = stationary_distribution(
        markov_matrix,
        increase_power=increase_power,
        normalized=False,
    )

    return scores


def _power_method(transition_matrix, increase_power=True, max_iter=10000):
    eigenvector = np.ones(len(transition_matrix))

    if len(eigenvector) == 1:
        return eigenvector

    transition = transition_matrix.transpose()

    for _ in range(max_iter):
        eigenvector_next = np.dot(transition, eigenvector)

        if np.allclose(eigenvector_next, eigenvector):
            return eigenvector_next

        eigenvector = eigenvector_next

        if increase_power:
            transition = np.dot(transition, transition)

    logger.warning("Maximum number of iterations for power method exceeded without convergence!")
    return eigenvector_next


def connected_nodes(matrix):
    _, labels = connected_components(matrix)

    groups = []

    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)

    return groups


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    # normalize probability distribution differently if we have negative transition values
    if np.min(weights_matrix) <= 0:
        return softmax(weights_matrix, axis=1)

    return weights_matrix / row_sum


def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = np.zeros(weights_matrix.shape)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1

    return create_markov_matrix(discrete_weights_matrix)


def stationary_distribution(
    transition_matrix,
    increase_power=True,
    normalized=True,
):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)

    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        distribution[group] = eigenvector

    if normalized:
        distribution /= n_1

    return distribution

This function use the degree centrality scores to generate an extracted summary

In [ ]:
"""
This example uses LexRank (https://www.aaai.org/Papers/JAIR/Vol22/JAIR-2214.pdf)
to create an extractive summarization of a long document.
The document is splitted into sentences using NLTK, then the sentence embeddings are computed. We
then compute the cosine-similarity across all possible sentence pairs.
We then use LexRank to find the most central sentences in the document, which form our summary.
Input document: First section from the English Wikipedia Section
Output summary:
Located at the southern tip of the U.S. state of New York, the city is the center of the New York metropolitan area, the largest metropolitan area in the world by urban landmass.
New York City (NYC), often called simply New York, is the most populous city in the United States.
Anchored by Wall Street in the Financial District of Lower Manhattan, New York City has been called both the world's leading financial center and the most financially powerful city in the world, and is home to the world's two largest stock exchanges by total market capitalization, the New York Stock Exchange and NASDAQ.
New York City has been described as the cultural, financial, and media capital of the world, significantly influencing commerce, entertainment, research, technology, education, politics, tourism, art, fashion, and sports.
If the New York metropolitan area were a sovereign state, it would have the eighth-largest economy in the world.
"""

def get_extracted_summary(model, stories, another_extractive_summary, story_name):
    '''
        Function that generate the extractive summaries of a story using lexrank and 'all-MiniLM-L6-v2' model.
        input:
            model:
                Pretrained sentence transformer 'all-MiniLM-L6-v2'.
            stories:
                All the original stories as dictionary with id2sentence, sentence2id, story. Look read_stories() function.
            another_extractive_summary:
                Extractive summary with nltk.sent_tokenize from the abstrct summary of the 'story_name'.
            story_name:
                Name of the story .txt
        output:
            extracted_summary:
                A list of couple: [number id of a sentence in the story, sentence of the story]
    '''
    n_sentence = len(another_extractive_summary)
    sentences = list(stories[story_name]["id2sentence"].values())
    
    #Compute the sentence embeddings
    embeddings = model.encode(sentences, convert_to_tensor=True)

    #Compute the pair-wise cosine similarities
    cos_scores = util.cos_sim(embeddings, embeddings).cpu().numpy()

    #Add all pairs to a list with their cosine similarity score
    all_sentence_combinations = []
    for i in range(len(cos_scores)-1):
        for j in range(i+1, len(cos_scores)):
            all_sentence_combinations.append([cos_scores[i][j], i, j])

    #Sort list by the highest cosine similarity score
    all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

    #Compute the centrality for each sentence
    centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

    #We argsort so that the first element is the sentence with the highest score
    most_central_sentence_indices = np.argsort(-centrality_scores)

    idxs = most_central_sentence_indices[:n_sentence]
    idxs.sort()
    extracted_summary = []
    for idx in idxs:
        extracted_summary.append([str(idx), sentences[idx].strip()])
    return extracted_summary

# Generate the extractive summaries

In [ ]:
# Read the stories from the datasets
stories = read_stories()
# Generate a splitted version of the abstract summaries.
nltk_summaries = generated_summary(stories)

100%|██████████| 54/54 [00:00<00:00, 421.47it/s, Read Short Story=bn_00283865n_The Great Carbuncle.txt]


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

100%|██████████| 91/91 [01:01<00:00,  1.48it/s, Generate summary of story=bn_00283865n_The Great Carbuncle.txt]


In [ ]:
# Generate restored extractive summaries.
# This is used to compute the number of sentences to mantain in the next extractive summary function.
nltk_extracted_summaries = {}
for story_name, story_dict in stories.items():
    nltk_extracted_summaries[story_name] = restore_sentences(stories[story_name], nltk_summaries[story_name], story_name)

In [ ]:
# Generate the extractive summaries using 'all-MiniLM-L6-v2' and take the number calculated above as the central n sentences.
extracted_summaries = {}
model = SentenceTransformer('all-MiniLM-L6-v2')
batch_pbar = tqdm(nltk_extracted_summaries.items(), total=len(list((nltk_extracted_summaries.items()))))
for story_name, nltk_extracted_summary in batch_pbar:
    batch_pbar.set_postfix({'Extractive Summarize': story_name})
    extracted_summaries[story_name] = get_extracted_summary(model, stories, nltk_extracted_summary, story_name)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

100%|██████████| 91/91 [00:12<00:00,  7.36it/s, Extractive Summarize=bn_00283865n_The Great Carbuncle.txt]


In [ ]:
# Save the results
    # Entry: DOCUMENT_ID SENTENCES_IDS
with open('results.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    for story_name, extracted_summary in extracted_summaries.items():
        DOCUMENT_ID = story_name[:-4]
        SENTENCE_IDS = ",".join(np.array(list(extracted_summaries[story_name]))[:, 0])
        writer.writerow([DOCUMENT_ID, SENTENCE_IDS])